In [1]:
import pandas as pd
import requests as rq
import numpy as np
import pyarrow
import os
from dotenv import load_dotenv
import time
from datetime import datetime
import sys #permite navegar por el sistema
sys.path.append("../") #solo aplica al soporte
from thefuzz import fuzz
from src.etl import transform as tr #con jupyter
from src.etl import extract as ex #con jupyter
import warnings
from pandas.errors import SettingWithCopyWarning


In [2]:
load_dotenv()

True

In [6]:
#importo los df
RUTA_SERVICE = os.getenv("RUTA_SERVICE")
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_VIAJES')
ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS = os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS')
ARCHIVO_GUARDAR_CONTINENTES = os.getenv('ARCHIVO_GUARDAR_CONTINENTES')
ARCHIVO_GUARDAR_OPCIONES_VIAJES =os.getenv('ARCHIVO_GUARDAR_OPCIONES_VIAJES')
ARCHIVO_GUARDAR_DATOS_API=os.getenv('ARCHIVO_GUARDAR_DATOS_API')
df_continentes = pd.read_pickle(ARCHIVO_GUARDAR_CONTINENTES)
df_destinos= pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_DESTINOS)
df_viajes= pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES)
df_opciones = pd.read_pickle(ARCHIVO_GUARDAR_OPCIONES_VIAJES)
df_turismo_emisor = pd.read_csv(ARCHIVO_GUARDAR_DATOS_API,delimiter=';',encoding='latin1')

In [7]:
df_turismo_emisor

,AÑO,MES,CCAA_ORIGEN,CONTINENTE_DESTINO,PAIS_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,2019,7,Andalucía,América,Argentina,"419,0","3616,0","8,6"
1,2019,7,Andalucía,América,Bolivia,"61,0","486,0","8,0"
2,2019,7,Andalucía,América,Brasil,"277,0","2214,0","8,0"
3,2019,7,Andalucía,América,Canadá,"483,0","6159,0","12,8"
4,2019,7,Andalucía,América,Chile,"242,0","2068,0","8,5"
...,...,...,...,...,...,...,...,...
81658,2024,12,Total Nacional,África,Seychelles,"146,0","1105,0","7,6"
81659,2024,12,Total Nacional,África,Sudáfrica,"986,0","9019,0","9,1"
81660,2024,12,Total Nacional,África,Tanzania,"590,0","5237,0","8,9"
81661,2024,12,Total Nacional,África,Túnez,"975,0","5012,0","5,1"


# Limpieza fichero turismo emisor

In [8]:
df_turismo_emisor.head()

,AÑO,MES,CCAA_ORIGEN,CONTINENTE_DESTINO,PAIS_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,2019,7,Andalucía,América,Argentina,"419,0","3616,0","8,6"
1,2019,7,Andalucía,América,Bolivia,"61,0","486,0","8,0"
2,2019,7,Andalucía,América,Brasil,"277,0","2214,0","8,0"
3,2019,7,Andalucía,América,Canadá,"483,0","6159,0","12,8"
4,2019,7,Andalucía,América,Chile,"242,0","2068,0","8,5"


In [9]:
#dejo solo los datos concretos a un pais:
indice_otros= df_turismo_emisor[df_turismo_emisor.PAIS_DESTINO.str.contains('Otros')].index.tolist()
indice_total=df_turismo_emisor[df_turismo_emisor.PAIS_DESTINO.str.contains('Total')].index.tolist()
indices_eliminar = indice_otros+indice_total
indices_eliminar
df_turismo_emisor.drop(indices_eliminar,inplace=True)

In [10]:
#Reemplazo los valores que al unirlo con el DF de escrapeo da lugar a duplicados:
df_turismo_emisor.PAIS_DESTINO= df_turismo_emisor.PAIS_DESTINO.replace(
    {'Antigua y Barbuda': 'Antigua',
    'Estados Unidos de América':'Estados Unidos',
    'Bhután': 'Bután',
    'Corea':'Corea Del Sur',
    'Zimbabwe':'Zimbabue'})

In [11]:
#guardo el fichero procesado
ARCHIVO_GUARDAR_DATOS_API_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_DATOS_API_PROCESADOS')
df_turismo_emisor.to_csv(ARCHIVO_GUARDAR_DATOS_API_PROCESADOS)

# Limpieza tabla continente

In [8]:
df_continentes_reducido = df_continentes[['continente', 'pais']].drop_duplicates()
df_continentes_reducido

,continente,pais
0,Europa,Albania
1,Europa,Alemania
2,Caribe,Antigua
3,América,Argentina
4,Caribe,Aruba
...,...,...
1075,Oriente Medio,Armenia
1437,Oriente Medio,Arabia Saudí
1498,Europa,Laponia
2186,Europa,Bosnia y Herzegovina


In [9]:
#compruebo si hay paises con más de un continente asociado
for pais in df_continentes_reducido.pais:
    df_pais = df_continentes_reducido[df_continentes_reducido['pais'] == pais]
    contador_continentes = df_pais.continente.value_counts()
    if len(contador_continentes)>1:
        print(f' para {pais} hay {len(contador_continentes)} continentes')

In [10]:
df_continentes.continente.unique()

array(['Europa', 'Caribe', 'América', 'Oceanía', 'África', 'Asia',
       'Oriente Medio', 'Islas Exóticas'], dtype=object)

## Obtención de los continentes correctos de los paises en Caribe, Oriente Medio e Islas Exóticas
Como existen continentes que realmente no son, obtengo los continentes correctos para que los datos en la BBDD sean correctos a través de la api OpenCage

In [12]:
#creación de la api key
API_KEY= os.getenv("API_KEY")
ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE')

In [24]:
ARCHIVO_GUARDAR_TOTAL_CIUDADES_API=os.getenv('ARCHIVO_GUARDAR_TOTAL_CIUDADES_API')

In [25]:
df_paises_api_todas_ciudades= pd.read_pickle(ARCHIVO_GUARDAR_TOTAL_CIUDADES_API)

In [26]:
df_paises_api_todas_ciudades

,ciudad,continente_api,pais_api,latitud,longitud
0,Tirana,Europe,Albania,41.328148,19.818444
1,Dürres,Europe,Albania,41.313255,19.446235
2,Berat,Europe,Albania,40.701864,19.958076
3,Kruja,Europe,Albania,41.511251,19.792852
4,Shkodër,Europe,Albania,42.068137,19.512144
...,...,...,...,...,...
1471,Mesa Verde,North America,Estados Unidos de América,33.605858,-114.731072
1472,Great Sand,Oceania,Australia,-38.262747,144.775661
1473,Dunes,Europe,Francia,44.087932,0.769998
1474,Pueblo,North America,Estados Unidos de América,38.263995,-104.614187


In [15]:
df_geolocalizacion_correcta_anterior

,pais,continente_api,pais_api,latitud,longitud,ciudad
0,Antigua,América,Antigua y Barbuda,17.103715,-61.79045,None
1,Aruba,América,Aruba,12.501363,-69.961848,None
2,Bahamas,América,Bahamas,24.773655,-78.000055,None
3,Barbados,América,Barbados,13.150033,-59.52503,None
4,Curaçao,América,Curazao,12.117649,-68.930926,None
...,...,...,...,...,...,...
380,NaN,Información desconocida,Información desconocida,Información desconocida,Información desconocida,Yllas
381,NaN,Europa,Eslovenia,46.67196,15.92369,Parque Nacional De Plitvice
382,NaN,Europa,Alemania,51.225402,6.776314,Dusseldorf
383,NaN,Información desconocida,Información desconocida,Información desconocida,Información desconocida,Delta De Okavango


In [12]:
#obtengo los paises/islas que no están asociados a los continentes reales:
lista_paises_islas = df_continentes_reducido[~df_continentes_reducido.continente.isin(['Europa', 'América', 'Oceanía', 'África', 'Asia'])].pais.tolist()
print(len(lista_paises_islas))
if len(lista_paises_islas)>0:
    #obtengo los datos ya obtenidos de la api de geolocalización
    df_continentes_correctos_anterior= pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE)
    df_continentes_correctos_anterior_filtrado=df_continentes_correctos_anterior[df_continentes_correctos_anterior['pais'].notna()]
    #incorporo la información en la tabla continentes original:
    ex.incorporar_información_df_original(df_continentes, df_continentes_correctos_anterior_filtrado,
                                      'pais', 'continente_api', 'continente', 
                                      df_continentes.continente.isin(['Caribe', 'Oriente Medio', 'Islas Exóticas']))
    df_continentes_reducido = df_continentes[['continente', 'pais']].drop_duplicates()
    lista_paises_islas = df_continentes_reducido[df_continentes_reducido.continente.isin(['Caribe', 'Oriente Medio', 'Islas Exóticas'])].pais.tolist()
    print(len(lista_paises_islas))
    if len(lista_paises_islas)>0:
        #obtengo los datos de la api
        df_continentes_correctos = pd.DataFrame()
        df_continentes_correctos['pais']=lista_paises_islas
        df_continentes_correctos[['continente_api','pais_api', 'latitud', 'longitud']]=(
            df_continentes_correctos.apply(lambda fila:pd.Series(
                tr.obtener_pais_continente_lat_long(fila['pais'], API_KEY)), axis=1)
        )
        #sustituyo los valores dados en Inglés por los valores en Español
        df_continentes_correctos.continente_api= df_continentes_correctos.continente_api.replace({'North America':'América',
                                                        'South America': 'América',
                                                        'Africa': 'África',
                                                        'Europe': 'Europa',
                                                        'Oceania':'Oceanía'})
        ex.incorporar_información_df_original(df_continentes, df_continentes_correctos,
                                      'pais', 'continente_api', 'continente', 
                                      df_continentes.continente.isin(['Caribe', 'Oriente Medio', 'Islas Exóticas']))
        df_continentes_correctos['ciudad']=None
        df_continentes_agrupado_geolocalizacion_api = pd.concat([df_continentes_correctos_anterior,df_continentes_correctos],axis=0) #uno los 2 DF de la api de geolocalizacion
    else:
        df_continentes_agrupado_geolocalizacion_api=df_continentes_correctos_anterior
    df_continentes_agrupado_geolocalizacion_api.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE)

34
0


In [13]:
df_continentes.continente.unique()

array(['Europa', 'América', 'Oceanía', 'África', 'Asia'], dtype=object)

In [14]:
df_continentes.head()

,continente,pais,fecha_escrapeo,en_ultimo_escrapeo
0,Europa,Albania,2025-04-14,Si
1,Europa,Alemania,2025-04-14,Si
2,América,Antigua,2025-04-14,Si
3,América,Argentina,2025-04-14,Si
4,América,Aruba,2025-04-14,Si


In [15]:
ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS=os.getenv("ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS")
df_continentes.to_pickle(ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS)

## Unión de los paises y continentes tanto escrapeados como obtenidos de la API de turismo emisor

In [16]:
df_turismo_emisor_procesado=pd.read_csv(ARCHIVO_GUARDAR_DATOS_API_PROCESADOS)

In [17]:
df_turismo_emisor_procesado.columns

Index(['Unnamed: 0', 'AÑO', 'MES', 'CCAA_ORIGEN', 'CONTINENTE_DESTINO',
       'PAIS_DESTINO', 'TURISTAS', 'PERNOCTACIONES', 'ESTANCIA_MEDIA'],
      dtype='object')

In [18]:
df_turismo_emisor_procesado.head()

,Unnamed: 0,AÑO,MES,CCAA_ORIGEN,CONTINENTE_DESTINO,PAIS_DESTINO,TURISTAS,PERNOCTACIONES,ESTANCIA_MEDIA
0,0,2019,7,Andalucía,América,Argentina,"419,0","3616,0","8,6"
1,1,2019,7,Andalucía,América,Bolivia,"61,0","486,0","8,0"
2,2,2019,7,Andalucía,América,Brasil,"277,0","2214,0","8,0"
3,3,2019,7,Andalucía,América,Canadá,"483,0","6159,0","12,8"
4,4,2019,7,Andalucía,América,Chile,"242,0","2068,0","8,5"


In [19]:
df_paises_continentes_turismos_emisor=df_turismo_emisor_procesado[['PAIS_DESTINO','CONTINENTE_DESTINO']]

In [20]:
df_paises_continentes_turismos_emisor.CONTINENTE_DESTINO.unique()

array(['América', 'Asia', 'Europa', 'Oceanía', 'África'], dtype=object)

In [21]:
df_paises_continentes_escrapeo = pd.read_pickle(ARCHIVO_GUARDAR_CONTINENTES_PROCESADOS)

In [22]:
df_paises_continentes_escrapeo=df_paises_continentes_escrapeo[['pais','continente']]

In [23]:
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.drop_duplicates()
df_paises_continentes_turismos_emisor=df_paises_continentes_turismos_emisor.drop_duplicates()

In [24]:
df_paises_continentes_turismos_emisor

,PAIS_DESTINO,CONTINENTE_DESTINO
0,Argentina,América
1,Bolivia,América
2,Brasil,América
3,Canadá,América
4,Chile,América
...,...,...
39985,Djibouti,África
40484,Trinidad y Tobago,América
41885,Guyana,América
48538,Yemen,Asia


In [25]:
df_paises_continentes_escrapeo.columns
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.rename(
    columns={'pais':'nombre_pais_destino', 'continente':'nombre_continente'})
df_paises_continentes_escrapeo= df_paises_continentes_escrapeo.sort_values(by='nombre_pais_destino').reset_index(drop=True)

In [26]:
df_paises_continentes_turismos_emisor.columns
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.rename(
    columns={'PAIS_DESTINO':'nombre_pais_destino', 'CONTINENTE_DESTINO':'nombre_continente'})
df_paises_continentes_turismos_emisor= df_paises_continentes_turismos_emisor.sort_values(by='nombre_pais_destino').reset_index(drop=True)

In [27]:
df_paises_continentes_totales=pd.concat(
    [df_paises_continentes_escrapeo,df_paises_continentes_turismos_emisor], axis=0)

In [28]:
df_paises_continentes_totales = df_paises_continentes_totales.drop_duplicates(['nombre_pais_destino', 'nombre_continente'])

In [29]:
df_paises_continentes_totales

,nombre_pais_destino,nombre_continente
0,Albania,Europa
1,Alemania,Europa
2,Antigua,América
3,Arabia Saudí,Asia
4,Argentina,América
...,...,...
147,Turquía,Europa
149,Ucrania,Europa
153,Venezuela,América
155,Yemen,Asia


# CREACIÓN TABLA ÚNICA VIAJES CON LOS VIAJES ESCRAPEADOS Y LAS OPCIONES

In [30]:
df_viajes.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36013 entries, 0 to 36012
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                36013 non-null  object        
 1   nombre_viaje        36013 non-null  object        
 2   duracion_viaje      36013 non-null  object        
 3   itinerario          36013 non-null  object        
 4   precio              36013 non-null  int64         
 5   url_viaje           35991 non-null  object        
 6   fecha_escrapeo      36013 non-null  datetime64[ns]
 7   en_ultimo_escrapeo  36013 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 2.2+ MB


In [31]:
df_opciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21505 entries, 0 to 21504
Data columns (total 11 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   pais                 21505 non-null  object        
 1   nombre_viaje         21505 non-null  object        
 2   opcion               21505 non-null  object        
 3   precio               21505 non-null  int64         
 4   url_viaje_opcion     21501 non-null  object        
 5   fecha_escrapeo       21505 non-null  datetime64[ns]
 6   en_ultimo_escrapeo   21505 non-null  object        
 7   url_viaje            21505 non-null  object        
 8   nombre_viaje_opcion  21505 non-null  object        
 9   duracion_viaje       21317 non-null  object        
 10  itinerario           21317 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(9)
memory usage: 1.8+ MB


## Extracción de los viajes correctos de la tabla viajes

In [32]:
#elimino los viajes que tienen la palabra opcion ya que luego incorporaré las opciones como viaje
indices_viajes_opciones = df_viajes[df_viajes.url_viaje.str.contains('opciones',na=False)].index.tolist()

In [33]:
df_viajes=df_viajes.drop(indices_viajes_opciones)
df_viajes


,pais,nombre_viaje,duracion_viaje,itinerario,precio,url_viaje,fecha_escrapeo,en_ultimo_escrapeo
0,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,https://es.tui.com/viaje/albania-y-montenegro,2025-04-07,Si
1,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,https://viajeonline.es.tui.com/es/idea/1463025...,2025-04-07,Si
2,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,https://viajeonline.es.tui.com/es/idea/7897512...,2025-04-07,Si
3,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,https://viajeonline.es.tui.com/es/idea/8404511...,2025-04-07,Si
4,Alemania,Fly & Drive Tirol al Completo,8 Días / 7 Noches,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",894,https://viajeonline.es.tui.com/es/idea/7906930...,2025-04-07,Si
...,...,...,...,...,...,...,...,...
36007,Zimbabue,Tras la Senda de los Elefantes,16 Días / 13 Noches,"Cataratas Victoria, Matobo, Botswana, Makgadik...",4675,https://es.tui.com/viaje/tras-la-senda-de-los-...,2025-05-14,Si
36008,Zimbabue,Río Perdido Clásico,16 Días / 13 Noches,"Cataratas Victoria, Valle de los Elefantes, P....",4930,https://es.tui.com/viaje/rio-perdido-clasico,2025-05-14,Si
36009,Zimbabue,Ruta del Okavango Premium,21 Días / 18 Noches,"Windhoek, Desierto de Namibia, Sossusvlei, Ses...",5335,https://es.tui.com/viaje/ruta-del-okavango-pre...,2025-05-14,Si
36010,Zimbabue,Maravillas del África Austral,16 Días / 13 Noches,"Windhoek, Desierto de Namibia, Cataratas Victo...",5895,https://es.tui.com/viaje/maravillas-del-africa...,2025-05-14,Si


## Crear un df similar al de viaje con las opciones

In [34]:
df_viajes.columns

Index(['pais', 'nombre_viaje', 'duracion_viaje', 'itinerario', 'precio',
       'url_viaje', 'fecha_escrapeo', 'en_ultimo_escrapeo'],
      dtype='object')

In [35]:
df_opciones.columns

Index(['pais', 'nombre_viaje', 'opcion', 'precio', 'url_viaje_opcion',
       'fecha_escrapeo', 'en_ultimo_escrapeo', 'url_viaje',
       'nombre_viaje_opcion', 'duracion_viaje', 'itinerario'],
      dtype='object')

In [36]:
df_viajes_opciones= df_opciones[['pais', 'nombre_viaje_opcion', 'duracion_viaje', 'itinerario', 'precio',
       'url_viaje_opcion', 'fecha_escrapeo', 'en_ultimo_escrapeo']]
df_viajes_opciones = df_viajes_opciones.rename(columns={'nombre_viaje_opcion':'nombre_viaje','url_viaje_opcion':'url_viaje'})

In [37]:
df_viajes_opciones.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21505 entries, 0 to 21504
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                21505 non-null  object        
 1   nombre_viaje        21505 non-null  object        
 2   duracion_viaje      21317 non-null  object        
 3   itinerario          21317 non-null  object        
 4   precio              21505 non-null  int64         
 5   url_viaje           21501 non-null  object        
 6   fecha_escrapeo      21505 non-null  datetime64[ns]
 7   en_ultimo_escrapeo  21505 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 1.3+ MB


## Agrupacion de los dos DF para crear un df viajes unico

In [38]:
df_viajes_agrupados = pd.concat([df_viajes,df_viajes_opciones],axis=0).reset_index(drop=True)

In [39]:
df_viajes_agrupados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49314 entries, 0 to 49313
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                49314 non-null  object        
 1   nombre_viaje        49314 non-null  object        
 2   duracion_viaje      49126 non-null  object        
 3   itinerario          49126 non-null  object        
 4   precio              49314 non-null  int64         
 5   url_viaje           49288 non-null  object        
 6   fecha_escrapeo      49314 non-null  datetime64[ns]
 7   en_ultimo_escrapeo  49314 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 3.0+ MB


### Corrección de duplicados

In [40]:
#Duplicados con mismo nombre y diferente url
dict_cambios_url= {"nombre_viaje":[],'url_anterior':[], 'url_nueva':[], 'url_anterior_en_ultimo_escrapeo':[], 
                   'url_nueva_en_ultimo_escrapeo':[]}
for viaje in df_viajes_agrupados.nombre_viaje.unique():
    df_filtrado = df_viajes_agrupados[df_viajes_agrupados.nombre_viaje == viaje]
    if len(df_filtrado.url_viaje.unique()) >1:
        url1= df_filtrado.url_viaje.unique()[-2]
        url2=df_filtrado.url_viaje.unique()[-1]
        #print(viaje)
        #print(len(df_filtrado.en_ultimo_escrapeo .unique()))
        if len(df_filtrado.en_ultimo_escrapeo.unique())>1 and len(df_filtrado.duracion_viaje.unique())==1 and (len(df_filtrado.itinerario.unique()))==1:
            en_ultimo_escrapeo_anterior = df_filtrado.en_ultimo_escrapeo.unique()[-2]
            en_ultimo_escrapeo_nuevo = df_filtrado.en_ultimo_escrapeo.unique()[-1]
            similitud = fuzz.ratio(url1, url2)
            #print(similitud)
            if similitud >74:
                dict_cambios_url['nombre_viaje'].append(viaje)
                dict_cambios_url['url_anterior'].append(url1)
                dict_cambios_url['url_nueva'].append(url2)
                dict_cambios_url['url_anterior_en_ultimo_escrapeo'].append(en_ultimo_escrapeo_anterior)
                dict_cambios_url['url_nueva_en_ultimo_escrapeo'].append(en_ultimo_escrapeo_nuevo)
            else:
                print(viaje)
                print(len(df_filtrado.url_viaje.unique()))
                print(url1)
                print(url2)
                print(similitud)
                display(df_filtrado.drop_duplicates(["pais", 'nombre_viaje', 'duracion_viaje', 'itinerario','url_viaje']))
#print(dict_cambios_url)
df_cambios_url = pd.DataFrame(dict_cambios_url)
df_cambios_url

,nombre_viaje,url_anterior,url_nueva,url_anterior_en_ultimo_escrapeo,url_nueva_en_ultimo_escrapeo
0,Boston y Nueva York,https://viajeonline.es.tui.com/es/idea/2225200...,https://viajeonline.es.tui.com/es/idea/2225200...,No,Si
1,Nueva York y Niágara,https://viajeonline.es.tui.com/es/idea/1632339...,https://viajeonline.es.tui.com/es/idea/2474016...,No,Si
2,Este Americano,https://viajeonline.es.tui.com/es/idea/1154208...,https://viajeonline.es.tui.com/es/idea/2631797...,No,Si
3,Fantasías del Oeste,https://viajeonline.es.tui.com/es/idea/1626061...,https://viajeonline.es.tui.com/es/idea/2493711...,No,Si
4,Ruta de la Música,https://viajeonline.es.tui.com/ES/idea/1626675...,https://viajeonline.es.tui.com/es/idea/2494554...,No,Si
5,Japón Fascinante y Ise y Kumano,https://viajeonline.es.tui.com/es/idea/1781411...,https://viajeonline.es.tui.com/es/idea/2502099...,No,Si
6,Mongolia Inédita,https://viajeonline.es.tui.com/es/idea/1018063...,https://viajeonline.es.tui.com/es/idea/2608480...,No,Si
7,Joyas del Este,https://viajeonline.es.tui.com/es/idea/2490414...,https://viajeonline.es.tui.com/es/idea/2490414...,No,Si
8,Este Mágico,https://viajeonline.es.tui.com/es/idea/2441141...,https://viajeonline.es.tui.com/es/idea/2488419...,No,Si
9,Oeste Americano y San Francisco,https://viajeonline.es.tui.com/es/idea/1451515...,https://viajeonline.es.tui.com/es/idea/1451515...,No,Si


In [41]:
ex.incorporar_información_df_original(df_viajes_agrupados, df_cambios_url,
                                      'nombre_viaje', 'url_nueva', 'url_viaje', 
                                      df_viajes_agrupados.url_viaje.isin(dict_cambios_url['url_anterior']))
ex.incorporar_información_df_original(df_viajes_agrupados, df_cambios_url,
                                      'nombre_viaje', 'url_nueva_en_ultimo_escrapeo', 'en_ultimo_escrapeo', 
                                      df_viajes_agrupados.url_viaje.isin(dict_cambios_url['url_anterior']))

In [42]:
#Duplicados con misma url y diferente nombre
dict_cambios_nombre_viaje= {"url_viaje":[],'nombre_anterior':[], 'nombre_nuevo':[]}
for url in df_viajes_agrupados.url_viaje.unique():
    df_filtrado = df_viajes_agrupados[df_viajes_agrupados.url_viaje == url]
    if len(df_filtrado.nombre_viaje.unique()) >1:
        #print(url)
        #print(len(df_filtrado.nombre_viaje.unique()))
        nombre1= df_filtrado.nombre_viaje.unique()[-2]
        nombre2=df_filtrado.nombre_viaje.unique()[-1]
        #print(nombre1)
        #print(nombre2)
        dict_cambios_nombre_viaje['url_viaje'].append(url)
        dict_cambios_nombre_viaje['nombre_anterior'].append(nombre1)
        dict_cambios_nombre_viaje['nombre_nuevo'].append(nombre2)
df_cambios_nombre_viaje = pd.DataFrame(dict_cambios_nombre_viaje)
df_cambios_nombre_viaje

,url_viaje,nombre_anterior,nombre_nuevo
0,https://viajeonline.es.tui.com/es/idea/1452125...,Tibet Express,Capitales Chinas y Tíbet
1,https://es.tui.com/viaje/oeste-a-tu-alcance,Oeste a tu alcance,Oeste a tu Alcance
2,https://viajeonline.es.tui.com/es/idea/1626555...,Nueva York desde Faro - Salida 08 Septiembre,Nueva York desde Faro
3,https://viajeonline.es.tui.com/es/idea/1220996...,Fly & Drive Oeste Americano y Hawai,Fly & Drive Oeste Americano y Hawái
4,https://viajeonline.es.tui.com/es/idea/1867627...,Oahu - Kauai - Big island (Hawai),Oahu - Kauai - Big island (Hawái)
5,https://viajeonline.es.tui.com/es/idea/1862292...,Oahu - Big island (Hawai) - Maui,Oahu - Big island (Hawái) - Maui
6,https://viajeonline.es.tui.com/es/idea/1872287...,Lo mejor de Alaska,Lo Mejor de Alaska
7,https://viajeonline.es.tui.com/es/idea/2497113...,Costa Rica en Fin de Año,Costa Rica con Manuel Antonio en Fin de Año


In [43]:
ex.incorporar_información_df_original(df_viajes_agrupados, df_cambios_nombre_viaje,
                                      'url_viaje', 'nombre_nuevo', 'nombre_viaje', 
                                      df_viajes_agrupados.nombre_viaje.isin(dict_cambios_nombre_viaje['nombre_anterior']))

In [44]:
df_viajes_agrupados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49314 entries, 0 to 49313
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                49314 non-null  object        
 1   nombre_viaje        49314 non-null  object        
 2   duracion_viaje      49126 non-null  object        
 3   itinerario          49126 non-null  object        
 4   precio              49314 non-null  int64         
 5   url_viaje           49288 non-null  object        
 6   fecha_escrapeo      49314 non-null  datetime64[ns]
 7   en_ultimo_escrapeo  49314 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 3.0+ MB


In [45]:
#elimino los que tiene valores nulos en duracion_Viaje y precio, porque no tengo forma de obtenerlo
#  ya que son viajes antiguos y las urls ya no están disponibles
df_viajes_agrupados=df_viajes_agrupados.dropna().reset_index(drop=True)

In [46]:
df_viajes_agrupados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49103 entries, 0 to 49102
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   pais                49103 non-null  object        
 1   nombre_viaje        49103 non-null  object        
 2   duracion_viaje      49103 non-null  object        
 3   itinerario          49103 non-null  object        
 4   precio              49103 non-null  int64         
 5   url_viaje           49103 non-null  object        
 6   fecha_escrapeo      49103 non-null  datetime64[ns]
 7   en_ultimo_escrapeo  49103 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(6)
memory usage: 3.0+ MB


In [47]:
#Limpio la columna itinerario de cara a la futura extracción de las ciudades
df_viajes_agrupados.itinerario = df_viajes_agrupados.itinerario.str.replace('.','')
#reemplazo los paréntesis y las ' y ' y ' e ' por una coma para dividir todas las ciudades:
df_viajes_agrupados['itinerario_modificado_para_dividir'] = df_viajes_agrupados.itinerario.str.replace(r'[\(\)]| y | e ', ' , ', regex=True)

In [48]:
#guardo el df resultante, con todos los viajes:
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS')
df_viajes_agrupados.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS)

# Desglose itinerario para obtener las ciudades

In [49]:
df_viajes_agrupados=pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS)
df_itinerarios = df_viajes_agrupados[['pais','itinerario_modificado_para_dividir']].drop_duplicates().reset_index(drop=True)
df_itinerarios

,pais,itinerario_modificado_para_dividir
0,Albania,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova..."
1,Alemania,Berlín
2,Alemania,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H..."
3,Alemania,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B..."
4,Alemania,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,..."
...,...,...
1392,Georgia,"Erevan, Etchmiadzin, Zvartnots, Sevan, Dilijan..."
1393,Laponia,Laponia
1394,Estados Unidos,"Los Ángeles, Las Vegas, Gran Cañón, Monument V..."
1395,Finlandia,Levi


In [50]:
df_itinerarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1397 entries, 0 to 1396
Data columns (total 2 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   pais                                1397 non-null   object
 1   itinerario_modificado_para_dividir  1397 non-null   object
dtypes: object(2)
memory usage: 22.0+ KB


In [51]:
df_viajes_agrupados.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 49103 entries, 0 to 49102
Data columns (total 9 columns):
 #   Column                              Non-Null Count  Dtype         
---  ------                              --------------  -----         
 0   pais                                49103 non-null  object        
 1   nombre_viaje                        49103 non-null  object        
 2   duracion_viaje                      49103 non-null  object        
 3   itinerario                          49103 non-null  object        
 4   precio                              49103 non-null  int64         
 5   url_viaje                           49103 non-null  object        
 6   fecha_escrapeo                      49103 non-null  datetime64[ns]
 7   en_ultimo_escrapeo                  49103 non-null  object        
 8   itinerario_modificado_para_dividir  49103 non-null  object        
dtypes: datetime64[ns](1), int64(1), object(7)
memory usage: 3.4+ MB


In [52]:
df_itinerarios=df_itinerarios.drop_duplicates()

In [53]:
df_itinerarios.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1397 entries, 0 to 1396
Data columns (total 2 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   pais                                1397 non-null   object
 1   itinerario_modificado_para_dividir  1397 non-null   object
dtypes: object(2)
memory usage: 22.0+ KB


In [54]:
df_itinerarios.itinerario_modificado_para_dividir.unique().tolist()

['Tirana, Dürres, Berat, Kruja, Shkodër, Petrovac, Budva, Cetinje , Bar',
 'Berlín',
 'Salzburgo, Gmunden, Linz, Melk, Viena, Graz, Hallstatt, St Wolfgang, St Johann in Tirol , Innsbruck',
 'Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, Bonn, Königswinter, Coblenza, Boppard, Cochem, Tréveris, Luxemburgo, Dinant, Gante, Brujas, Lovaina, Malinas , Amberes',
 'sbruck, Cataratas Krimml, Vipiteno, Bresanona, Bolzano, Trento, Marmolada, Canazei, Cortina D’Ampezzo , Brunico',
 'Hamburgo, Stade, Bremen, Luneburg, Schwerin, Wismar , Lubeck',
 'Frankfurt, Baden-Baden, Estrasburgo, Selva Negra, Titisee, Friburgo, lago Constanza, Riquewihr, Colmar , Heidelberg',
 'Buenos Aires, El Calafate , Iguazú',
 'Buenos Aires, Ushuaia, El Calafate , Iguazú',
 'Buenos Aires, Puerto Madryn, El Calafate , Iguazú',
 'Buenos Aires, El Calafate, Iguazú , Río de Janeiro',
 'Buenos Aires, Ushuaia, El Calafate, Bariloche , Iguazú',
 'Buenos Aires, Puerto Madryn, Ushuaia, El Calafate , Iguazú',
 'Buenos Aires, Bariloch

In [55]:
def conversion_itineario_en_lista (valor):
    valor = valor.split(',')  # convertir string en lista, separando por comas
    valor = [v.strip() for v in valor]  # opcional: quitar espacios extra
    return valor

In [56]:
df_itinerarios['lista_itinerario']= df_itinerarios.itinerario_modificado_para_dividir.apply(conversion_itineario_en_lista)
df_itinerarios

,pais,itinerario_modificado_para_dividir,lista_itinerario
0,Albania,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...","[Tirana, Dürres, Berat, Kruja, Shkodër, Petrov..."
1,Alemania,Berlín,[Berlín]
2,Alemania,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...","[Salzburgo, Gmunden, Linz, Melk, Viena, Graz, ..."
3,Alemania,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...","[Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, ..."
4,Alemania,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...","[sbruck, Cataratas Krimml, Vipiteno, Bresanona..."
...,...,...,...
1392,Georgia,"Erevan, Etchmiadzin, Zvartnots, Sevan, Dilijan...","[Erevan, Etchmiadzin, Zvartnots, Sevan, Dilija..."
1393,Laponia,Laponia,[Laponia]
1394,Estados Unidos,"Los Ángeles, Las Vegas, Gran Cañón, Monument V...","[Los Ángeles, Las Vegas, Gran Cañón, Monument ..."
1395,Finlandia,Levi,[Levi]


## Extracción de la ciudad en aquellos itinerarios que solo aparecen 1 vez:

In [ ]:
dict_pais_correcto_itinerario = {'itinerario_modificado_para_dividir':[],'lista_itinerario':[], 'pais_correcto':[]}
for itinerario_modificado in df_itinerarios.itinerario_modificado_para_dividir.unique().tolist():
    #print(itinerario)
    df_filtrado = df_itinerarios[df_itinerarios.itinerario_modificado_para_dividir == itinerario_modificado]
    #display(df_filtrado)
    if len(df_filtrado.pais.unique())==1:
        lista_itinerario = df_filtrado['lista_itinerario'].tolist()[0]
        dict_pais_correcto_itinerario['itinerario_modificado_para_dividir'].append(itinerario_modificado)
        dict_pais_correcto_itinerario['lista_itinerario'].append(lista_itinerario)
        dict_pais_correcto_itinerario['pais_correcto'].append(df_filtrado.pais.unique()[0])
df_pais_correcto_itinerario=pd.DataFrame(dict_pais_correcto_itinerario)
df_pais_correcto_itinerario

In [ ]:
#divido las ciudades para asignar el pais a las ciudades:
df_pais_correcto_itinerario= df_pais_correcto_itinerario.explode('lista_itinerario') # Explode para deshacer las listas en filas
df_pais_correcto_itinerario = df_pais_correcto_itinerario.rename(columns={'lista_itinerario': 'ciudad'}) # Renombro la columna a ciudad
df_pais_correcto_itinerario['ciudad'] = df_pais_correcto_itinerario['ciudad'].str.replace(r'\s+', ' ', regex=True).str.strip() #eliminar espacios de delante y duplicados entre dos palabras
df_pais_correcto_itinerario


In [ ]:
#compruebo si existen ciudades duplicadas:
df_ciudades_web = df_pais_correcto_itinerario[['ciudad','pais_correcto']]
df_ciudades= df_ciudades_web.drop_duplicates()
df_ciudades

In [ ]:
for ciudad in df_ciudades.ciudad.unique():
    df_filtrado = df_ciudades[df_ciudades.ciudad == ciudad]
    if len(df_filtrado.pais_correcto.unique()) >1:
        print(f'La ciudad {ciudad} aparece en los siguientes paises{df_filtrado.pais_correcto.unique()}')


Hay algunas ciudades duplicadas, pero algunas de ellas sí que pertenecen a diferentes paises, por tanto lo dejo como está

## Extracción de la ciudad en los itinerarios que aparecen en más de 1 pais

In [ ]:
df_itinerarios_duplicados_en_paises= df_itinerarios[
    ~df_itinerarios.itinerario_modificado_para_dividir.isin(df_pais_correcto_itinerario.itinerario_modificado_para_dividir.tolist())]

In [ ]:
df_itinerarios_duplicados_en_paises= df_itinerarios_duplicados_en_paises.explode('lista_itinerario') # Explode para deshacer las listas en filas
df_itinerarios_duplicados_en_paises = df_itinerarios_duplicados_en_paises.rename(columns={'lista_itinerario': 'ciudad'}) # Renombro la columna a ciudad
df_itinerarios_duplicados_en_paises['ciudad'] = df_itinerarios_duplicados_en_paises['ciudad'].str.replace(r'\s+', ' ', regex=True).str.strip() #eliminar espacios de delante y duplicados entre dos palabras
df_itinerarios_duplicados_en_paises=df_itinerarios_duplicados_en_paises.reset_index(drop=True)

In [ ]:
df_itinerarios_duplicados_en_paises['pais_correcto'] = None

In [ ]:
ex.incorporar_información_df_original(df_itinerarios_duplicados_en_paises, df_pais_correcto_itinerario,'ciudad','pais_correcto', 'pais_correcto' )

In [ ]:
df_itinerarios_duplicados_en_paises

In [ ]:
df_itinerarios_duplicados_en_paises.info()

In [ ]:
df_itinerarios_duplicados_sin_pais= df_itinerarios_duplicados_en_paises[pd.isnull(df_itinerarios_duplicados_en_paises.pais_correcto)].reset_index(drop=True)
df_itinerarios_duplicados_sin_pais

In [ ]:
lista_ciudades_sin_paises = df_itinerarios_duplicados_sin_pais.ciudad.unique().tolist()
len(lista_ciudades_sin_paises)


In [ ]:
df_itinerarios_duplicados_en_paises

In [ ]:
#obtengo los paises de las ciudades que aparecen en itinerarios en varios paises:
lista_ciudades_sin_paises = df_itinerarios_duplicados_en_paises[pd.isnull(df_itinerarios_duplicados_en_paises.pais_correcto)].reset_index(drop=True).ciudad.unique().tolist()
print(len(lista_ciudades_sin_paises))
if len(lista_ciudades_sin_paises)>0:
    #obtengo los datos ya obtenidos de la api de geolocalización
    df_geolocalizacion_correcta_anterior= pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE).reset_index(drop=True)
    df_geolocalizacion_correcta_anterior_filtrado= df_geolocalizacion_correcta_anterior[df_geolocalizacion_correcta_anterior['ciudad'].notna()]
    #incorporo la información en la tabla continentes original:
    ex.incorporar_información_df_original(df_itinerarios_duplicados_en_paises, df_geolocalizacion_correcta_anterior_filtrado,
                                      'ciudad', 'pais_api', 'pais_correcto',
                                      pd.isnull(df_itinerarios_duplicados_en_paises.pais_correcto))
    lista_ciudades_sin_paises= df_itinerarios_duplicados_en_paises[pd.isnull(df_itinerarios_duplicados_en_paises.pais_correcto)].reset_index(drop=True).ciudad.unique().tolist()
    print(len(lista_ciudades_sin_paises))
    if len(lista_ciudades_sin_paises)>0:
        #obtengo los datos de la api
        df_paises_correctos = pd.DataFrame()
        df_paises_correctos['ciudad']=lista_ciudades_sin_paises
        df_paises_correctos[['continente_api','pais_api', 'latitud', 'longitud']]=(
            df_paises_correctos.apply(lambda fila:pd.Series(
                tr.obtener_pais_continente_lat_long(fila['ciudad'], API_KEY)), axis=1))
        #sustituyo los valores dados en Inglés por los valores en Español
        df_paises_correctos.continente_api= df_paises_correctos.continente_api.replace({'North America':'América',
                                                        'South America': 'América',
                                                        'Africa': 'África',
                                                        'Europe': 'Europa',
                                                        'Oceania':'Oceanía'})
        ex.incorporar_información_df_original(df_itinerarios_duplicados_en_paises, df_paises_correctos,
                                        'ciudad', 'pais_api', 'pais_correcto', 
                                        pd.isnull(df_itinerarios_duplicados_en_paises.pais_correcto))
        df_agrupado_geolocalizacion_api = pd.concat([df_geolocalizacion_correcta_anterior,df_paises_correctos],axis=0) #uno los 2 DF de la api de geolocalizacion
    else:
        df_agrupado_geolocalizacion_api=df_geolocalizacion_correcta_anterior
    df_agrupado_geolocalizacion_api.to_pickle(ARCHIVO_GUARDAR_ESCRAPEO_API_OPENCAGE)

In [ ]:
df_agrupado_geolocalizacion_api

## Creo el df itinerario para guardarlo para la posterior fase de carga

In [ ]:
df_pais_correcto_itinerario

In [ ]:
df_itinerarios_duplicados_en_paises = df_itinerarios_duplicados_en_paises[['itinerario_modificado_para_dividir','ciudad','pais_correcto']]
df_itinerarios_duplicados_en_paises

In [ ]:
df_itinerario_ciudades = pd.concat([df_pais_correcto_itinerario,df_itinerarios_duplicados_en_paises], axis=0)
df_itinerario_ciudades

In [ ]:
#Reemplazo los valores que al unirlo con el DF de escrapeo da lugar a duplicados:
df_itinerario_ciudades.pais_correcto= df_itinerario_ciudades.pais_correcto.replace(
    {'Estados Unidos de América':'Estados Unidos'})

# Creo una formula para dividir las ciudades desde el DF original y asignarle el valor del pais de la página web solo cuando el pais coincida con el de la api

In [ ]:
ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS=os.getenv('ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS')
df_itinerario_ciudades.to_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS)

In [7]:
ARCHIVO_GUARDAR_TOTAL_CIUDADES_API = os.getenv('ARCHIVO_GUARDAR_TOTAL_CIUDADES_API')
API_KEY = os.getenv('API_KEY')
ARCHIVO_EXTRACCION_API_DUPLICADOS = os.getenv('ARCHIVO_EXTRACCION_API_DUPLICADOS')
ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS = os.getenv('ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS')


In [8]:
df_viajes_agrupados = pd.read_pickle(ARCHIVO_GUARDAR_ESCRAPEO_VIAJES_PROCESADOS)

In [9]:
df_viajes_agrupados

,pais,nombre_viaje,duracion_viaje,itinerario,precio,url_viaje,fecha_escrapeo,en_ultimo_escrapeo,itinerario_modificado_para_dividir
0,Albania,Albania y Montenegro,8 Días / 7 Noches,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova...",1755,https://es.tui.com/viaje/albania-y-montenegro,2025-04-07,Si,"Tirana, Dürres, Berat, Kruja, Shkodër, Petrova..."
1,Alemania,Berlín,4 Días / 3 Noches,Berlín,708,https://viajeonline.es.tui.com/es/idea/1463025...,2025-04-07,Si,Berlín
2,Alemania,"Fly & Drive Austria, Entre Lagos y Montañas",8 Días / 7 Noches,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H...",827,https://viajeonline.es.tui.com/es/idea/7897512...,2025-04-07,Si,"Salzburgo, Gmunden, Linz, Melk, Viena, Graz, H..."
3,Alemania,Fly & Drive Carlomagno a su aire,10 Días / 9 Noches,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B...",857,https://viajeonline.es.tui.com/es/idea/8404511...,2025-04-07,Si,"Bruselas, Durbuy, Lieja, Aquisgrán, Colonia, B..."
4,Alemania,Fly & Drive Tirol al Completo,8 Días / 7 Noches,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,...",894,https://viajeonline.es.tui.com/es/idea/7906930...,2025-04-07,Si,"sbruck, Cataratas Krimml, Vipiteno, Bresanona,..."
...,...,...,...,...,...,...,...,...,...
49098,Zimbabue,Cataratas y Parques de Botsuana y Mauricio,13 Días / 10 Noches,"Cataratas Victoria, PN Chobe, Delta de Okavang...",5369,https://es.tui.com/viaje/cataratas-y-parques-d...,2025-05-14,Si,"Cataratas Victoria, PN Chobe, Delta de Okavang..."
49099,Zimbabue,Cataratas y Parques de Botsuana y Mozambique,13 Días / 10 Noches,"Cataratas Victoria, PN Chobe, Delta de Okavang...",5868,https://es.tui.com/viaje/cataratas-y-parques-d...,2025-05-14,Si,"Cataratas Victoria, PN Chobe, Delta de Okavang..."
49100,Zimbabue,Cataratas y Parques de Botsuana y Maldivas,13 Días / 9 Noches,"Cataratas Victoria, PN Chobe, Delta de Okavang...",6189,https://es.tui.com/viaje/cataratas-y-parques-d...,2025-05-14,Si,"Cataratas Victoria, PN Chobe, Delta de Okavang..."
49101,Zimbabue,Zimbabue Salvaje,11 Días / 8 Noches,"Cataratas Victoria, PN Hwange y Mana Pools",6539,https://es.tui.com/viaje/zimbabue-salvaje,2025-05-14,Si,"Cataratas Victoria, PN Hwange , Mana Pools"


In [8]:
#tr.desglosar_ciudades_itinerarios_2(df_viajes_agrupados,API_KEY,ARCHIVO_GUARDAR_TOTAL_CIUDADES_API, ARCHIVO_EXTRACCION_API_DUPLICADOS)

In [ ]:
ARCHIVO_EXTRACCION_API_DUPLICADOS = os.getenv('ARCHIVO_EXTRACCION_API_DUPLICADOS')


In [ ]:
df_paises_api_ciudades_duplicadas= pd.read_pickle(ARCHIVO_EXTRACCION_API_DUPLICADOS)
df_paises_api_ciudades_duplicadas.columns
#df_paises_api_ciudades_duplicadas= df_paises_api_ciudades_duplicadas[~pd.isnull(df_paises_api_ciudades_duplicadas['continente_api'])]
#df_paises_api_ciudades_duplicadas.to_pickle(ARCHIVO_EXTRACCION_API_DUPLICADOS)
df_paises_api_ciudades_duplicadas

,pais,ciudad,continente_api,pais_api,latitud,longitud,resultado
24,Alemania,Colonia,North America,Honduras,14.108469,-87.231085,Ciudad Y Pais Encontrado
31,Alemania,Luxemburgo,Europe,Alemania,52.506983,13.3511,Ciudad Y Pais Encontrado
38,Alemania,Sbruck,Europe,Alemania,51.5,10.5,Ciudad Y Pais Encontrado
60,Alemania,Friburgo,Europe,Alemania,47.99609,7.8494,Ciudad Y Pais Encontrado
61,Alemania,Lago Constanza,Europe,Alemania,47.647769,9.34718,Ciudad Y Pais Encontrado
...,...,...,...,...,...,...,...
2993,Panama,San Blas En Velero,North America,Panama,9.0,-80.0,Ciudad Y Pais Encontrado
3027,Sudafrica,Reserva De Kapama,Africa,Sudafrica,-29.0,24.0,Ciudad Y Pais Encontrado
3042,Taiwan,Monasterio Fo Guang Shan,Asia,Taiwan,24.0,121.0,Ciudad Y Pais Encontrado
3109,Estados Unidos,Moab Arches,North America,Estados Unidos De America,38.600884,-109.576167,Ciudad Y Pais Encontrado


In [ ]:
def desglosar_ciudades_itinerarios_2 (df_viajes_agrupados,API_KEY,ARCHIVO_GUARDAR_TOTAL_CIUDADES_API, ARCHIVO_EXTRACCION_API_DUPLICADOS):
    df_itinerarios = df_viajes_agrupados[['pais','itinerario_modificado_para_dividir']].drop_duplicates().reset_index(drop=True) #elimino duplicados
    df_itinerarios['lista_itinerario']= df_itinerarios.itinerario_modificado_para_dividir.apply(tr.conversion_itineario_en_lista) #separo las ciudades
    df_itinerarios_ciudades = df_itinerarios.explode('lista_itinerario').reset_index(drop=True) #divido el DF por ciudades
    df_itinerarios_ciudades=df_itinerarios_ciudades.rename(columns={'lista_itinerario':'ciudad'}) #renombro la columna a ciudad

    #normalizo y capitalizo la columna ciudad:
    df_itinerarios_ciudades['ciudad'] = df_itinerarios_ciudades['ciudad'].map(tr.normalizar_texto)
    df_itinerarios_ciudades['ciudad'] = df_itinerarios_ciudades['ciudad'].map(tr.capitalizar_texto)

    indices_sin_ciudades = df_itinerarios_ciudades[df_itinerarios_ciudades.ciudad == ''].index #creo una lista con los indices de las filas sin ciudad
    #elimino las filas sin ciudad:
    df_itinerarios_ciudades= df_itinerarios_ciudades.drop(indices_sin_ciudades).reset_index(drop=True)
    #obtengo los datos de la api extraidos anteriormente
    df_paises_api_todas_ciudades= pd.read_pickle(ARCHIVO_GUARDAR_TOTAL_CIUDADES_API)# '../data/data_raw/geolocalizacion_api_total_ciudades.pkl'

    #normalizo y capitalizo la columna ciudad
    df_paises_api_todas_ciudades['ciudad'] = df_paises_api_todas_ciudades['ciudad'].map(tr.normalizar_texto)
    df_paises_api_todas_ciudades['ciudad'] = df_paises_api_todas_ciudades['ciudad'].map(tr.capitalizar_texto)

    df_itinerarios_ciudades['pais_api']=None #creo la columna pais_api para incorporar la información de la api
    ex.incorporar_información_df_original(df_itinerarios_ciudades,df_paises_api_todas_ciudades,'ciudad', 'pais_api', 'pais_api')
    df_itinerarios_ciudades_sin_lista = df_itinerarios_ciudades[['pais', 'ciudad', 'pais_api']] # me quedo solo con pais, ciudad y pais api
    df_itinerarios_ciudades_sin_lista=df_itinerarios_ciudades_sin_lista.drop_duplicates().reset_index(drop=True) #elimino duplicados

    #Normalizo la columna paises:
    df_itinerarios_ciudades['pais'] = df_itinerarios_ciudades['pais'].map(tr.normalizar_texto)
    df_itinerarios_ciudades['pais'] = df_itinerarios_ciudades['pais'].map(tr.capitalizar_texto)
    df_itinerarios_ciudades['pais_api'] = df_itinerarios_ciudades['pais_api'].map(tr.normalizar_texto)
    df_itinerarios_ciudades['pais_api'] = df_itinerarios_ciudades['pais_api'].map(tr.capitalizar_texto)

    #comparo si el pais de la api coincide con el pais del viaje:
    df_itinerarios_ciudades_sin_lista['resultado']= None
    for i,row in df_itinerarios_ciudades_sin_lista.iterrows():
        if row['pais'] == row["pais_api"]:
            df_itinerarios_ciudades_sin_lista.loc[i,'resultado'] ='Pais correcto'
        else:
            df_itinerarios_ciudades_sin_lista.loc[i,'resultado'] ='Pais no coincide'

    #creo DF con los paises correctos  (pais viaje = pais api):
    df_itinerarios_ciudades_sin_lista_paises_correctos = df_itinerarios_ciudades_sin_lista[df_itinerarios_ciudades_sin_lista.resultado=='Pais correcto']
    
    print(f'{len(df_itinerarios_ciudades_sin_lista_paises_correctos)} paises correctos')
    #creo una lista de ciudades con paises correctas para añadir la información al DF
    lista_ciudades_pais_correcto = df_itinerarios_ciudades_sin_lista_paises_correctos['ciudad'].unique().tolist()
 

    df_itinerarios_ciudades_sin_lista['resultado_2']= None
    for i,row in df_itinerarios_ciudades_sin_lista.iterrows():
        if row['ciudad'] in lista_ciudades_pais_correcto:
            df_itinerarios_ciudades_sin_lista.loc[i,'resultado_2'] ='Pais_correcto_identificado'
        else:
            df_itinerarios_ciudades_sin_lista.loc[i,'resultado_2'] ='Pais a identificar por API'

    #incorporo la informacion de los paises correctos al df de ciudades paises inicial
    df_itinerarios_ciudades_sin_lista['pais_final']=None #creo la columna con el pais definitivo
    ex.incorporar_información_df_original(df_itinerarios_ciudades_sin_lista,df_itinerarios_ciudades_sin_lista_paises_correctos,
                                          'ciudad','pais','pais_final',df_itinerarios_ciudades_sin_lista.resultado_2=='Pais_correcto_identificado')
    print(f'Se ha asignado el pais a las ciudades cuyo pais de la api de la ciudad coincide con el pais del viaje: {len(lista_ciudades_pais_correcto)}')


    #me quedo con la parte del DF que hay que identificar por API:
    df_itinerarios_a_identificar_por_api =df_itinerarios_ciudades_sin_lista[df_itinerarios_ciudades_sin_lista.resultado_2=='Pais a identificar por API']

    #importo el df con los datos extraidos de la api anteriormente:
    df_paises_api_ciudades_duplicadas= pd.read_pickle(ARCHIVO_EXTRACCION_API_DUPLICADOS)
    
    ex.incorporar_información_df_original(df_itinerarios_ciudades_sin_lista, df_paises_api_ciudades_duplicadas,
                                            'ciudad', 'pais_api', 'pais_final', 
                                            df_itinerarios_ciudades_sin_lista.resultado_2=='Pais a identificar por API')
    print(f'Se ha asignado el pais a las ciudades cuyo pais de la api de la ciudad no coincide con el pais del viaje: {len(df_paises_api_ciudades_duplicadas)}')


    #me quedo con la parte del DF que hay que identificar por API final:
    df_itinerarios_a_identificar_por_api = df_itinerarios_ciudades_sin_lista[pd.isnull(df_itinerarios_ciudades_sin_lista.pais_final)] #COMPROBADO HASTA AQUI
    warnings.simplefilter(action='ignore', category=SettingWithCopyWarning)
    #Extraigo los datos de la API:
    if len(df_itinerarios_a_identificar_por_api)>0:
        print(len(df_itinerarios_a_identificar_por_api))
        #df_itinerarios_ciudades_sin_duplicados = pd.DataFrame()
        df_itinerarios_ciudades_sin_duplicados=df_itinerarios_a_identificar_por_api[['pais','ciudad']].copy()
        df_itinerarios_ciudades_sin_duplicados= df_itinerarios_ciudades_sin_duplicados.drop_duplicates()
        #columnas_destino = ['continente_api','pais_api', 'latitud', 'longitud','resultado']
        #for col in columnas_destino:
            #df_itinerarios_ciudades_sin_duplicados[col] = None

        #df_itinerarios_ciudades_sin_duplicados.loc[:,columnas_destino]=(
        resultados_api = df_itinerarios_ciudades_sin_duplicados.apply(
                lambda fila:pd.Series(
                    tr.obtener_pais_continente_lat_long_con_pais(fila['ciudad'],fila['pais'], API_KEY),
                    index=['continente_api','pais_api', 'latitud', 'longitud','resultado']),
                    axis=1)
        
        # Concatenar los resultados al DataFrame original
        df_itinerarios_ciudades_sin_duplicados = pd.concat([df_itinerarios_ciudades_sin_duplicados, resultados_api], axis=1)

        print(f'Se ha extraido la informacion de: {len(df_itinerarios_ciudades_sin_duplicados)} ciudades de la API')



        #normalizo y capitalizo el texto
        for col in df_itinerarios_ciudades_sin_duplicados.columns:
            df_itinerarios_ciudades_sin_duplicados[col] = df_itinerarios_ciudades_sin_duplicados[col].map(tr.normalizar_texto)
            df_itinerarios_ciudades_sin_duplicados[col] = df_itinerarios_ciudades_sin_duplicados[col].map(tr.capitalizar_texto)

        df_itinerarios_a_identificar_por_api_nuevo = pd.concat([df_paises_api_ciudades_duplicadas,df_itinerarios_ciudades_sin_duplicados], axis=0)
        df_itinerarios_a_identificar_por_api_nuevo.to_pickle(ARCHIVO_EXTRACCION_API_DUPLICADOS)

        ex.incorporar_información_df_original(df_itinerarios_ciudades_sin_lista,
                                             df_itinerarios_ciudades_sin_duplicados,'ciudad','pais_api','pais_final',
                                        pd.isnull(df_itinerarios_ciudades_sin_lista.pais_final))
        print(f'Se ha asignado el pais a las ciudades con la información extraida de la API:{len(df_itinerarios_ciudades_sin_duplicados)}')

    #Reemplazo los valores que al unirlo con el DF de escrapeo da lugar a duplicados:
    df_itinerarios_ciudades_sin_lista.pais_final= df_itinerarios_ciudades_sin_lista.pais_final.replace(
            {'Estados Unidos de América':'Estados Unidos',
             'Spain': 'España',
             'Arabia Saudita': 'Arabia Saudí',
            'Fiyi': 'Fiji',
            'Francia': 'France',
            'Islas Turcas y Caicos': 'Turks and Caicos',
            'Papua-Nueva Guinea': 'papua nueva guinea',
            'Zimbawe': 'Zimbabue',
            'Chipre Del Norte': 'Chipre',
            'Chipre del Norte / Chipre': 'Chipre'})

    #incorporo la información al df_itinerarios_original
    df_itinerarios_ciudades_completo = df_itinerarios_ciudades[['itinerario_modificado_para_dividir', 'ciudad']]
    df_itinerarios_ciudades_completo['pais_correcto']= None
    ex.incorporar_información_df_original(df_itinerarios_ciudades_completo, df_itinerarios_ciudades_sin_lista,
                                           'ciudad','pais_final','pais_correcto')
    print(f'Se ha informado el pais correcto a cada ciudad de cada itinerario:{len(df_itinerarios_ciudades_completo)}')


    ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS_1=os.getenv('ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS_1')
    df_itinerarios_ciudades_sin_lista.to_pickle(ARCHIVO_GUARDAR_ITINERARIOS_PROCESADOS_1)
    print(f'Se ha guardado la informacion de itinerarios y ciudades:{len(df_itinerarios_ciudades_completo)} ')
    return df_itinerarios_ciudades_completo

In [77]:
df=desglosar_ciudades_itinerarios_2(df_viajes_agrupados,API_KEY,ARCHIVO_GUARDAR_TOTAL_CIUDADES_API, ARCHIVO_EXTRACCION_API_DUPLICADOS)
df.info()

1445 paises correctos
Se ha asignado el pais a las ciudades cuyo pais de la api de la ciudad coincide con el pais del viaje: 1445
Se ha asignado el pais a las ciudades cuyo pais de la api de la ciudad no coincide con el pais del viaje: 805
Se ha informado el pais correcto a cada ciudad de cada itinerario:9347
Se ha guardado la informacion de itinerarios y ciudades:9347 
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9347 entries, 0 to 9346
Data columns (total 3 columns):
 #   Column                              Non-Null Count  Dtype 
---  ------                              --------------  ----- 
 0   itinerario_modificado_para_dividir  9347 non-null   object
 1   ciudad                              9347 non-null   object
 2   pais_correcto                       9347 non-null   object
dtypes: object(3)
memory usage: 219.2+ KB
